---

# CSCI 3202, Spring 2022
# Homework 5
# Due: Friday April 15, 2022 at 6:00 PM

<br> 

### Your name: Kyle Moe

<br> 

---

Some useful packages and libraries:




In [27]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy as cp

---

# Hidden Markov Models
In this assignment, you will build a Hidden Markov Model from a data set to detect typos in text. One major difference between this assignment and previous assignments in this class is we're not providing any skeleton code, you need to generate your code from scratch. You can use any code provided during lecture or on previous assignments. 
<p><strong>Please read the entire assignment before beginning.</strong> 
    
<p>The main components of the assignment are the following:

### Part I - Building an HMM from data
1.  Implement a method to parse the data file and build an HMM from the data.
2.  Implement a method, Laplace smoothing, to smooth the transition and emission probabilities

### Part II - Implement the Viterbi algorithm
1.	Implement the Viterbi algorithm for finding the most likely sequence of states through the HMM, given "evidence"; and
2.	Run your code on a separate dataset and explore its performance.
    
### Part III - Summarize your results
    



## Part 1 - Building an HMM from data
The first part of the assignment is to build an HMM from data. Recall that an HMM involves a hidden state that changes over time, as well as observable evidence that is used to infer the hidden state. In our example from lecture, we used the observation of how a patient felt to infer whether they were healthy or sick. 

An HMM is defined by three sets of probabilities:
1.	For each state s, the probability of observing each output o at that state (P(E[t]=o | X[t]=s). These are the emission probabilities.
2.	From each state s, the probability of traversing to every other state s' in one time step (P(X[t+1]=s' | X[t]=s)). These are the transition probabilities.
3.	A distribution over the start state (P(X[0])). This is the initial state distribution.

For the start state distribution (P(X[0])), in this assignment, you will assume that there is a single dummy start state, distinct from all other states, and to which the HMM can never return.  When you implement the Viterbi algorithm in Part III, you will need to include the probability of making a transition from this dummy start state to each of the other states.

### Generate the conditional probability tables
To generate the transition and emission probability tables, use the data file called <em>typos20.data</em> on Canvas that contains sequences of state-output pairs, i.e., sequences of the form: <br>

x[1] e[1] <br>
x[2] e[2] <br>
. <br>
. <br>
. <br>
x[n] e[n]. <br>  

In this data, the x[i] is the state and the e[i] is the observation at time i. 

For instance, to estimate the probability of output o being observed in state s, you count the number of times that output o appears with state s in the given data, and divide by a normalization constant (so that the probabilities of all outputs from that state add up to one).  In this case, that normalization constant would simply be the number of times that state s appears at all in the data.

In this problem, state is letter that should have been typed, and output is the letter that was typed.  Given a sequence of outputs (i.e., actually typed letters), the problem is to reconstruct the hidden state sequence (i.e., the intended sequence of letters).  Thus, data for this problem looks like this:

i i <br>
n n <br>
t t <br>
r r <br>
o o <br>
d x <br>
u u <br>
c c <br>
t t <br>
i i <br>
o i <br>
n n <br>
_ _ <br>
t t <br>
h h <br>
e e <br>
_ _ <br>

where the left column is the correct text and the right column contains text with errors.

### Data
Data for this assignment was generated by starting with a text document, in this case, the Unabomber's Manifesto, which was chosen not for political reasons, but for its convenience being available on-line and of about the right length, converting all numbers and punctuation to white space and converting all letters to lower case.  The remaining text is a sequence only over the lower case letters and the space character, represented in the data files by an underscore character.  Next, typos were artificially added to the data as follows: with 80% probability, the correct letter is transcribed, but with 20% probability, a randomly chosen neighbor (on an ordinary physical keyboard) of the letter is transcribed instead.  Space characters are always transcribed correctly. 

As an example, the original document begins:

<em>introduction the industrial revolution and its consequences have been a disaster for the human race they have greatly increased the life expectancy of those of us who live in advanced countries but they have destabilized society have made life unfulfilling have subjected human beings to indignities have led to widespread psychological suffering in the third world to physical suffering as well and have inflicted severe damage on the natural world the continued development of technology will worsen the situation it will certainly subject human beings to greater indignities and inflict greater damage on the natural world it will probably lead to greater social disruption and psychological suffering and it may lead to increased physical suffering even in advanced countries the industrial technological system may survive or it may break down if it survives it may eventually achieve a low level of physical and psychological suffering but only after passing through a long and very painful period of adjustment and only at the cost of permanently reducing human beings and many other living organisms to engineered products and mere cogs in the social machine </em>

With 20% noise, it looks like this:

<em>introductipn the industfial revolhtjon and its consequences bafw newn a diszster rkr the yumab race thdy have grwatky increased the ljte esoectandy od thosr of is who libe in advanced coubfries but they have fewtabipuzee xociwty have made life ujfuorillkng have wubjwdted humah beints to incihbjtids have led to qidespreze lsyxhllotical shffeding kn tne third wkrld to phyxicql sufcefimg as weol and hqve ingoidtex srvere damsge on the natural world the confinued developmeng of twvhjllogy will wotsen thd situation it wull certaknly sunjrct yyman beingw tl greater ibdignities snd infpixt greagwr damsge on fhe natural alrld it wjlk probably lwad tk grezter sofiqp disrupgiln and pstchokofucal wufterkng anc it may kead fl uncreqxed pgusiczl sucfreinh even in acgajved countries the indhsteial tedhnologicak system may survivr or ut nay brezk down uf it survives it nay evenyuakly achieve a los lwvel of phyxkcal and psycyoligical sufveribg but only after passing theough a long amd very painful periox od adjuwtmebt and only at the fost kf permsnently reducing hymaj veings abs nsjy otgwr kuving orbanisms to envineered leoduxfs amd mere clgs in thr soxiap maxhjne </em>

The error rate (fraction of characters that are mistyped) is about 16.5% (less than 20% because space characters were not corrupted).

### Implement Laplace smoothing
When you generate the transition and emission probabilities, you will also need to smooth the estimates.  Here's the rationale, consider flipping a coin for which the probability of heads is p, where p is unknown, and our goal is to estimate p. We count how many times the coin comes up heads and divide by the total number of coin flips.  If we flip the coin 1000 times and it comes up heads 367 times, it is very reasonable to estimate p as approximately 0.367.  However, suppose we flip the coin only twice and we get heads both times.  Is it reasonable to estimate p as 1.0?  Intuitively, given that we only flipped the coin twice, we don’t have enough data to conclude that the coin will always come up heads. Smoothing is a way of avoiding these types of conclusions when we have limited data available.  

A simple smoothing method, called Laplace smoothing (or Laplace's law of succession or add-one smoothing in your textbook), is to estimate p by:

<p>
<center>$\frac{(1+number of heads)}{(2+number of flips)}$</center>

We add 1 to each output that could be observed in the numerator, and add the total number of states to the denominator.

If we are keeping count of the number of heads and the number of tails, this rule is equivalent to starting each of our counts at one, rather than zero.  This latter view generalizes to the case in which there are more than two possible outcomes (for instance, estimating the probability of a die coming up on each of its six faces). 
Another advantage of Laplace smoothing is that it avoids estimating any probabilities to be zero, even for events never observed in the data.  For HMMs, this is important since zero probabilities can be problematic for some algorithms.
    
For this assignment, you should use Laplace-smoothed estimates of probabilities.  For instance, returning to the problem of estimating the probability of output o being observed in state s, you would use one plus the number of times output o appears in state s in the given data, divided by a normalization constant. In this case, the normalization constant would be the number of times state s appears in the data, plus the total number of possible outputs. You will need to also work out Laplace-smoothed estimates for item 2, i.e., for the probability of making a transition from one state to another, as well as the probability of making a transition from the dummy start state to any of the other states. 


In [28]:
## Your code goes here.
## Your code needs to display the first 10 lines of your emission and transition tables, 
## and generate an output file with the complete tables, called Assignment8Output_Lastname_Firstname
## Your code also needs to be structured as an HMM class


In [29]:
trainingData = open('./typos20.data','r')
emissionCounts = {'a':{},'b':{},'c':{},'d':{},'e':{},
                  'f':{},'g':{},'h':{},'i':{},'j':{},
                  'k':{},'l':{},'m':{},'n':{},'o':{},
                  'p':{},'q':{},'r':{},'s':{},'t':{},
                  'u':{},'v':{},'w':{},'x':{},'y':{},'z':{},'_':{},'0':{}}

# Initializing to 1 automatically applies Laplace Smoothing
occuranceCounts = {'a':1,'b':1,'c':1,'d':1,'e':1, 
                   'f':1,'g':1,'h':1,'i':1,'j':1,
                   'k':1,'l':1,'m':1,'n':1,'o':1,
                   'p':1,'q':1,'r':1,'s':1,'t':1,
                   'u':1,'v':1,'w':1,'x':1,'y':1,'z':1,'_':1,'0':1}


for key in emissionCounts:
    emissionCounts[key] = cp.deepcopy(occuranceCounts)
    
transitionCounts = cp.deepcopy(emissionCounts)

prev = '0'

for line in trainingData:
    stateEmit = line.split()
    
    emissionCounts[stateEmit[0]][stateEmit[1]] += 1
    transitionCounts[prev][stateEmit[0]] += 1 # change to stateEmit[0] to use ground truth states
    prev = stateEmit[0]

In [30]:
for state in emissionCounts:
    occurrances = sum(emissionCounts[state].values())
    for emit in emissionCounts[state]:
        emissionCounts[state][emit] = np.log(emissionCounts[state][emit] / occurrances)
        
for state in transitionCounts:
    occurrances = sum(transitionCounts[state].values())
    for tran in transitionCounts[state]:
        transitionCounts[state][tran] = np.log(transitionCounts[state][tran] / occurrances)
        
print("Emission Table (log probabilities): \n")
print(emissionCounts)

print("\nTransition Table (log probabilities): \n")
print(transitionCounts)

Emission Table (log probabilities): 

{'a': {'a': -0.2297763191055673, 'b': -9.20109779060925, 'c': -9.20109779060925, 'd': -9.20109779060925, 'e': -9.20109779060925, 'f': -9.20109779060925, 'g': -9.20109779060925, 'h': -9.20109779060925, 'i': -9.20109779060925, 'j': -9.20109779060925, 'k': -9.20109779060925, 'l': -9.20109779060925, 'm': -9.20109779060925, 'n': -9.20109779060925, 'o': -9.20109779060925, 'p': -9.20109779060925, 'q': -2.684904714566286, 'r': -9.20109779060925, 's': -2.676068132765788, 't': -9.20109779060925, 'u': -9.20109779060925, 'v': -9.20109779060925, 'w': -9.20109779060925, 'x': -9.20109779060925, 'y': -9.20109779060925, 'z': -2.7210532286825972, '_': -9.20109779060925, '0': -9.20109779060925}, 'b': {'a': -7.5883236773352225, 'b': -0.22504409037218392, 'c': -7.5883236773352225, 'd': -7.5883236773352225, 'e': -7.5883236773352225, 'f': -7.5883236773352225, 'g': -3.0774641708183723, 'h': -3.1339763810817147, 'i': -7.5883236773352225, 'j': -7.5883236773352225, 'k': -7.5

In [31]:
states = ['a','b','c','d','e',
          'f','g','h','i','j',
          'k','l','m','n','o',
          'p','q','r','s','t',
          'u','v','w','x','y','z','_']
trans_p = transitionCounts
emit_p = emissionCounts

In [32]:
start_p = {'a': -3.332204510175204,
 'b': -3.332204510175204,
 'c': -3.332204510175204,
 'd': -3.332204510175204,
 'e': -3.332204510175204,
 'f': -3.332204510175204,
 'g': -3.332204510175204,
 'h': -3.332204510175204,
 'i': -3.332204510175204,
 'j': -3.332204510175204,
 'k': -3.332204510175204,
 'l': -3.332204510175204,
 'm': -3.332204510175204,
 'n': -3.332204510175204,
 'o': -3.332204510175204,
 'p': -3.332204510175204,
 'q': -3.332204510175204,
 'r': -3.332204510175204,
 's': -3.332204510175204,
 't': -3.332204510175204,
 'u': -3.332204510175204,
 'v': -3.332204510175204,
 'w': -3.332204510175204,
 'x': -3.332204510175204,
 'y': -3.332204510175204,
 'z': -3.332204510175204,
 '_': -3.332204510175204}


## Part II - Viterbi - finding the most likely sequence

Once your CPTs for your HMM are built, the second part of the assignment is to implement the Viterbi algorithm and use your CPTs to compute the most probable sequence of states (according to the HMM that you built from the data) for a given sequence of outputs.

There is another file on Moodle called *typos20Test.data* that contains test sequences of state-output pairs. You will provide your Viterbi code with just the output part of each of these sequences, and from this, you must compute the most likely sequence of states to produce the output sequence. The state part of these sequences is provided so that you can compare the estimated state sequences generated by your code to the actual state sequences that generated this data.  Note that these two sequences will not necessarily be identical, even if you have correctly implemented the Viterbi algorithm.

A numerical tip: the Viterbi algorithm involves multiplying many probabilities together.  Since each of these numbers is smaller than one (possibly much smaller), you can end up working with numbers that are tiny enough to be computationally indistinguishable from zero. To avoid this problem, it is recommended that you work with log probabilities. To do so, rather than multiplying two probabilities p and q, simply add their logarithms using the rule:
<p>
<center> log(pq) = log(p) + log(q) </center>

You will probably find that there is never a need to store or manipulate actual probabilities; instead, everything can be done using log probabilities.
The text reconstructed using an HMM with the Viterbi algorithm looks like this:
    <p>
*introduction the industrial revoluthon and its consequences bare neen a dissster ror the tuman race they have greatly increased the lite esoectandy od those of is who libe in advanced counfries but they have festabupusee cocisty have made live intiorilling have wibjested human beints to incingitids have led to widesprese lsysullotical suffeding in the third world to physical surcefing as weol and have ingoistes severe damage on the natural world the continued developmeng of techillogy will wotsen the situation it will certaknly sunirct tyman beinge tl greater indithities and infoist greager damage on the natural aleld it will probably owad to grester sofial distuption and pstchomofucal wiftering and it may kead fl increqxed ogusical suctreing even in achanved countries the industeial technologicak system may survive or ut nay break down if it survives it nay eventually achieve a los level of physical and psycholigical survering but only arter passing theough a long and very paindul perios od adjustment and only at the fost of permanently reducing human veings ans nany other kiving organisms to envineered leodusts and mere clys in the social machine*
        
The error rate has dropped to about 10.4%.


In [33]:
## Your code goes here. You need to implement the viterbi algorithm, and display the output that your code generates
## You also need to display the error rate for your algorithm.

In [43]:
obs = []
state = []
testData = open('./typos20Test.data','r')
for line in testData:
    stateEmit = line.split()
    obs.append(stateEmit[1])
    state.append(stateEmit[0])

In [44]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0] [st] = {"prob": start_p[st] + emit_p[st] [obs[0]], "prev": None} #
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = V[t - 1] [states[0]] ["prob"] + trans_p[states[0]] [st] #
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t - 1] [prev_st] ["prob"] + trans_p[prev_st] [st] #
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            max_prob = max_tr_prob + emit_p[st] [obs[t]] #
            V[t] [st] = {"prob": max_prob, "prev": prev_st_selected}

    #for line in dptable(V):
    #    print(line)

    opt = []
    max_prob = -100000000
    best_st = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1] [previous] ["prev"])
        previous = V[t + 1] [previous] ["prev"]
    
    print ("The steps of states are: " + " ".join(opt[0:1166]) + " (etc.) with highest (log) probability of %s" % max_prob)
    return opt

def dptable(V):
    # Print a table of steps from dictionary
    yield " " * 5 + "     ".join(("%3d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%lf" % v[state] ["prob"]) for v in V)

In [45]:
opt = viterbi(obs, states, start_p, trans_p, emit_p)

The steps of states are: i n t r o d u c t i o n _ t h e _ i n d u s t r i a l _ r e v o l u t h o n _ a n d _ i t s _ c o n s e q u e n c e s _ b a r e _ n e e n _ a _ d i s s s t e r _ r o r _ t h e _ t u m a n _ r a c e _ t h e y _ h a v e _ g r e a t l y _ i n c r e a s e d _ t h e _ l i t e _ e s o e c t a n d y _ o d _ t h o s e _ o f _ i s _ w h o _ l i b e _ i n _ a d v a n c e d _ c o u n f r i e s _ b u t _ t h e y _ h a v e _ f e s t a b u p u s e e _ c o c i s t y _ h a v e _ m a d e _ l i v e _ i n t i o r i l l i n g _ h a v e _ w i b j e s t e d _ h u m a n _ b e i n t s _ t o _ i n c i n g i t i d s _ h a v e _ l e d _ t o _ w i d e s p r e s e _ l s y s u l l o t i c a l _ s u f f e d i n g _ i n _ t h e _ t h i r d _ w o r l d _ t o _ p h y s i c a l _ s u r c e f i n g _ a s _ w e o l _ a n d _ h a v e _ i n g o i s t e s _ s e v e r e _ d a m a g e _ o n _ t h e _ n a t u r a l _ w o r l d _ t h e _ c o n t i n u e d _ d e v e l o p m e n g _ o f _ t e c h i l l o g

In [50]:
c = [i != j for i,j in zip(opt,state)]
errorRateTest = sum(c)/len(c)
print("Error Rate (Test): " + str(errorRateTest))

Error Rate (Test): 0.10431618819776714


## Part III - Summarize your results
### Purpose: 
The objective of this assignment is to apply the Viterbi Algorithm to Hidden Markov Model on a long sequence of noisy data. In this context, what this means is that we were to represent the sequential characters in text as a series of states where noise in the observed states requires us to predict what we think to be the ground truth text using the Viterbi Algorithm.
### Procedure:
##### Transition Probabilities
To generate the transition probabilities, I created a dictionary whose keys represent all the unique character states (including the dummy start character and underscore) whose values are dictionaries representing the probabilities of the succeeding character. This was generated by iterating through the lines of the file where tally of the current state's (X_i) occurances is incremented within the dictionary that the previous state X_{i-1} points to. After iterating through all the lines, each of these tallies is divided by the sum off all counts within the \'previous state\'s\' dictionary. Laplace smoothing requires only that all of these counts begin at 1 rather than 0.
##### Emission Probabilities
I used largely the same data structure of nested dictionaries for the emission probabilities as the transition probabilities. The main change is that rather than the dictionary's keys representing the previous state, they represent the current state and the keys within the dictionary that these keys point to give the probabilities of the corresponding emissions. This was also generated by iterating through the lines and counting occurances for those to then be divided by the sum of all emissions for one of the states.
##### Viterbi Algorithm
Within the Viterbi Algorithm the likely sequence of states is enumerated through a list of dictionaries corresponding to each observed state. These dictionaries hold keys for each of the possible character state who point to their own dictionaries containing the most likely transition probability and its corresponding previous state. These are generated by joining the likelihood of the previous state and its transition to the key's state. These previous states are then backtracked to select the likely sequence of transitions by beginning with the likeliest end state and its prior.
### Data
The data is parsed directly from the given data file where the ground truth state and observed output are listed adjacently and sequenced through sequential lines within the file. These states (and observations) can take any of the 27 characters when space is included. To parse this data, all that was requireed was to iterate through these lines while retaining the previous state (at least when generating the transition probabilities). By simply counting occurances, we can generate data structures that contain the occurance probabiltiies and transition probabilities using the saved previous states. For the test data, we again iterate through the lines and simply record the observed state within a list for the Viterbi Algorithm. The only pre-processing done was to remove the first line corresponding to the two periods whithin this file (the dummy start was handled with a different character).
### Results
For simplicity, discussion of the results will be focused to the above output corresponding to the introduction paragraph. We can see very clearly that for many of the words with only a few typos within, the Viterbi Algorithm does a rather good job of resolving the correct character. This can be seen early on as, for instance, the \'o\' in the \'introduction\' gets fixed from \'introductipn\' and \'industfial\' is fixed to \'industrial\'. However, with a change in \'disaster\' to \'diszster\' the algorithm can only produce \'dissster\'. This is because, while the sequences \'sz\' and \'zs\' are both infrequent transitions, \'ss\' is relatively frequent and the algorithm seems, in this case, to prefer this change than the change to \'sas\'.

There are certain cases where this algorithm consistently performs  poorly. If the randomization changed the first character of a given word, it struggled to make the necessary change. This can be seen in the sequence of words: \'bafw newn a diszster rkr the yumab\'. In this sentance, \'have\', \'been\', \'for\', and \'human\' have had changes applied to the first character. The output of the Viterbi Algorithm decided that this string is more likely to reflect the states of \'bare neen a dissster ror the tuman\'. Only the in the case of \'human\' is the first letter changed when incorrect and even in this case it is not able to change it to the correct state. This is because there is more parity among the transtion probabilties following the \'\_\' character than others so it is less equipped to fix such subsequent characters that begin the words. 

There are additional issues resolving characters that, in general, tend to be used less frequently. For example, the letter \'x\' very often follows the letter \'e\' and the one word in the above paragraph where an \'x\' would appear is in the word \'expectancy\' which appears as \'esoectandy\' with noise. Now, because \'x\' is simply more infrequent than other characters, the algorithm is poorly equipped to resolve it and leaves the character as \'s\'. This makes sense because even within the paragraph, the sequence \'es\' appear eight times, much more than \'ex\'.

Least surprisingly, the algorithm does poorly with words that are very noisy. The most apparent example may be \'destabilized\' which became \'fewtabipuzee\'. The Viterbi Algorithm can do very little to fix this and its best guess was that this word should be \'festabupusee\', again showing the consequences of changing the first character as well as the difficulty when dealing with a lot of noise. It also shows the limitations of only considering the adjacent characters and not where those characters appear in the word as few words end in \'ee\' but many end in \'ed\' yet this wasn't able to fix this issue. 